<a href="https://colab.research.google.com/github/vishusharma1914/pneumonia-detection-inceptionv3/blob/main/pneumonia_detection_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pneumonia Detection using InceptionV3
**Submitted by:** Vishu Sharma
**Date:** 08 July 2025


In [9]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, roc_auc_score, f1_score


In [10]:
# Load the PneumoniaMNIST dataset
path = tf.keras.utils.get_file(
    'pneumoniamnist.npz',
    'https://zenodo.org/record/6496656/files/pneumoniamnist.npz?download=1'
)
data = np.load(path)
x_train, y_train = data['train_images'], data['train_labels']
x_val, y_val = data['val_images'], data['val_labels']
x_test, y_test = data['test_images'], data['test_labels']

In [11]:
# Preprocess images: resize to 150x150 and convert grayscale to RGB
def preprocess_images(images):
    images = np.stack([images]*3, axis=-1)  # Convert to 3-channel
    images = tf.image.resize(images, [150, 150]).numpy()
    return images / 255.0

x_train = preprocess_images(x_train)
x_val = preprocess_images(x_val)
x_test = preprocess_images(x_test)

In [12]:
# Create the InceptionV3 model
base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
# Compute class weights to handle imbalance
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.flatten())
class_weights = {i: weight for i, weight in enumerate(class_weights)}

In [14]:
import tensorflow as tf

# Example preprocessing
x_train = tf.image.resize(x_train, [150, 150])
x_val = tf.image.resize(x_val, [150, 150])

# If images are grayscale (1 channel), convert to RGB (3 channels)
if x_train.shape[-1] == 1:
    x_train = tf.image.grayscale_to_rgb(x_train)
    x_val = tf.image.grayscale_to_rgb(x_val)


In [15]:
# Train the model
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=3,
    batch_size=32,
    class_weight=class_weights
)

Epoch 1/3
148/148 ━━━━━━━━━━━━━━━━━━━━ 344s 2s/step - accuracy: 0.7492 - loss: 0.6603 - val_accuracy: 0.8931 - val_loss: 0.2880
Epoch 2/3
148/148 ━━━━━━━━━━━━━━━━━━━━ 366s 2s/step - accuracy: 0.8858 - loss: 0.3268 - val_accuracy: 0.9198 - val_loss: 0.2172
Epoch 3/3
148/148 ━━━━━━━━━━━━━━━━━━━━ 307s 2s/step - accuracy: 0.8936 - loss: 0.2977 - val_accuracy: 0.9065 - val_loss: 0.2875


In [16]:
# Evaluate the model
preds = model.predict(x_test)
y_pred = (preds > 0.5).astype(int)

print(classification_report(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, preds))

20/20 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step
              precision    recall  f1-score   support

           0       0.84      0.77      0.80       234
           1       0.87      0.91      0.89       390

    accuracy                           0.86       624
   macro avg       0.85      0.84      0.85       624
weighted avg       0.86      0.86      0.86       624

F1 Score: 0.8897243107769424
AUC: 0.9253889984659215
